# Hier der Code um für einen Kanal alle Videos+Transcripts runterzuladen

### Abhängigkeiten

In [1]:
from urllib.request import urlopen
import json
import youtube_dl, pickle
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import YouTube
import os

### Funktionen

In [2]:
# Das Problem ist, dass der Username nicht unbedingt der Name ist unter dem der User auf Youtube aufgeführt ist.
# chess24 hat den Username chess24media --> das sieht man aber auf Youtube nirgends. 
# Eventuell die Channel-ID also anders holen, z.B.: https://commentpicker.com/youtube-channel-id.php
def get_channel_id(username):
    API_key = ''
    url=f"https://www.googleapis.com/youtube/v3/channels?key={API_key}&forUsername={USER_NAME}&part=id"
    inp = urlopen(url)
    resp = json.load(inp)
    
    return resp['items'][0]['id']


In [3]:
def get_video_ids(playlist_id):
    youtube_dl_options = {
        'skip_download': True,
        'ignoreerrors': True
    }
    with youtube_dl.YoutubeDL(youtube_dl_options) as ydl:
        videos = ydl.extract_info(f'https://www.youtube.com/channel/{playlist_id}/videos')
    return videos

def get_video_ids_old(playlist_id):
    with youtube_dl.YoutubeDL({'ignoreerrors': True}) as ydl:
        playd = ydl.extract_info(playlist_id, download=False)
        return playd

def save_playlist(username, playlist):
    with open('videos/'+username+'/playlist'+username+'.pickle', 'wb') as f:
        pickle.dump(playlist, f, pickle.HIGHEST_PROTOCOL)

def load_playlist(username):
    with open('videos/'+username+'/playlist'+username+'.pickle', 'rb') as f:
        playlist = pickle.load(f)
        return playlist

    

In [4]:
#video_id = playlist['entries'][100]['id']
def get_transcript(video_id):
    transscript = YouTubeTranscriptApi.get_transcript(video_id)
    return transscript

def save_transcript(transscript, video_id, user_name):
    with open('videos/'+user_name+'/'+video_id+'/transcript.pickle', 'wb') as f:
        pickle.dump(transscript, f, pickle.HIGHEST_PROTOCOL)

def no_transcript(video_id, user_name):
    if os.path.exists('videos/'+user_name+'/'+video_id+'/transcript.pickle'):
        return False
    return True
        
def load_transcript(video_id, user_name):
    with open('videos/'+user_name+'/'+video_id+'/transcript.pickle', 'rb') as f:
        transcript = pickle.load(f)
        return transcript

In [5]:
# playlist['entries'][100]['webpage_url']
def get_video(url, path):
    yt = YouTube(url)
    yt.streams.filter(file_extension='mp4', res="720p").first().download(path)
    
#def rename_video(video_id, video_title, username):
#    os.rename(username+'/'+video_title+'.mp4', username+'/'+video_id+'.mp4')

def no_video(video_id, username):
    try:
        for file in os.listdir('videos/'+username+'/'+video_id):
            if file.endswith('.mp4'):
                return False
    except:
        return True
    return True

def make_dir(path):
    try:
        os.mkdir(path)
    except:
        print(path, 'already exists.')

In [6]:
# playlist für den Channel laden und speichern:

USER_NAME = "chess24media"
PLAYLIST_ID = get_channel_id(USER_NAME)




In [7]:
# In diesen Ordner kommt alles rein
make_dir('videos/'+USER_NAME)

print(USER_NAME)
print(PLAYLIST_ID)

if 0:   # Das macht man nur beim ersten Mal
    playlist = get_video_ids(PLAYLIST_ID)
    save_playlist(USER_NAME, playlist)


videos/chess24media already exists.
chess24media
UCkTCNuQ2mGfW6-SpHpaze_g


In [8]:
playlist = load_playlist(USER_NAME)

print(len(playlist))


9


In [9]:
playlist['entries'][0]['duration']

19548

In [10]:
# Dann alles runterladen: 

counter = 0
loaded = 0

for video_id, video_title, video_url in [(entry['id'],entry['title'],entry['webpage_url']) for entry in playlist['entries']]:
    
    counter += 1

    if 'banter' in video_title or 'Banter' in video_title:
        print('loaded:',loaded)
        print()
        print(counter,video_title)
        print(video_id)
        
        make_dir('videos/'+USER_NAME+'/'+video_id)

        print('transcript?',no_transcript(video_id,  USER_NAME))
        print('video?',no_video(video_id, USER_NAME))

        if no_transcript(video_id,  USER_NAME):
            try:
                transscript = get_transcript(video_id)
                save_transcript(transscript,video_id, USER_NAME)
            except:
                print("Couldn't load the transcript.")

        if no_video(video_id, USER_NAME):
            try:
                get_video(video_url,'videos/'+USER_NAME+'/'+video_id)
                loaded += 1
            except:
                print("Couldn't load the video.")


loaded: 0

7 Banter Blitz with Jan Gustafsson
HEpnenp664Q
videos/chess24media/HEpnenp664Q already exists.
transcript? False
video? False
loaded: 0

10 Banter Blitz with Jan Gustafsson
mEgxx0BhUsM
videos/chess24media/mEgxx0BhUsM already exists.
transcript? False
video? False
loaded: 0

64 Banter Blitz with Ju Wenjun
nXlFuh2LV6M
videos/chess24media/nXlFuh2LV6M already exists.
transcript? False
video? False
loaded: 0

77 Banter Blitz with Sam Shankland | Play vs. 1.d4
JmT2W8QwYCQ
videos/chess24media/JmT2W8QwYCQ already exists.
transcript? False
video? False
loaded: 0

80 Banter Blitz with Sam Shankland
oEBy59OnG04
videos/chess24media/oEBy59OnG04 already exists.
transcript? False
video? False
loaded: 0

99 Banter Blitz with Nils Grandelius + AMA
EApbOn3p0Sc
videos/chess24media/EApbOn3p0Sc already exists.
transcript? False
video? False
loaded: 0

111 Banter Blitz with Midas Ratsma
_bXs0eSXWeY
videos/chess24media/_bXs0eSXWeY already exists.
transcript? False
video? False
loaded: 0

117 Bante

Couldn't load the transcript.
loaded: 0

544 Banter Blitz with Vladislav Artemiev
hIe6qsIzLTk
videos/chess24media/hIe6qsIzLTk already exists.
transcript? True
video? False
Couldn't load the transcript.
loaded: 0

546 Banter Blitz with Peter Leko
5-eGwi2NSmY
videos/chess24media/5-eGwi2NSmY already exists.
transcript? True
video? False
Couldn't load the transcript.
loaded: 0

553 Banter Blitz with Teimour Radjabov
MnKv3Qpe7jQ
videos/chess24media/MnKv3Qpe7jQ already exists.
transcript? False
video? False
loaded: 0

565 Banter Blitz with Teimour Radjabov
nRkr7wsQIUM
videos/chess24media/nRkr7wsQIUM already exists.
transcript? False
video? False
loaded: 0

575 Banter Blitz with Teimour Radjabov
TH1Uax8uPr0
videos/chess24media/TH1Uax8uPr0 already exists.
transcript? False
video? False
loaded: 0

580 Banter Blitz with Maxime Vachier-Lagrave
1EeXK7rMwx4
videos/chess24media/1EeXK7rMwx4 already exists.
transcript? False
video? False
loaded: 0

665 Banter Blitz with Wesley So
Q5bHm_ff3cs
videos/ch

Couldn't load the transcript.
loaded: 0

995 Banter Blitz with Teimour Radjabov
5cFHzGJvS1c
videos/chess24media/5cFHzGJvS1c already exists.
transcript? False
video? False
loaded: 0

996 Banter Blitz with Sam Shankland
zW_wEjs_yL4
videos/chess24media/zW_wEjs_yL4 already exists.
transcript? False
video? False
loaded: 0

1002 Banter Blitz with Peter Svidler
BN47DkLLIb4
videos/chess24media/BN47DkLLIb4 already exists.
transcript? False
video? False
loaded: 0

1005 Banter Blitz with Peter Svidler
ssBcIg3cEPI
videos/chess24media/ssBcIg3cEPI already exists.
transcript? False
video? False
loaded: 0

1007 Final | Jan-Krzysztof Duda vs. Liem Quang Le | San Fermin Banter Blitz Cup | Finals
GCNxwAfK0Ak
videos/chess24media/GCNxwAfK0Ak already exists.
transcript? False
video? False
loaded: 0

1008 Final | Liem Quang Le vs. Jan-Krzysztof Duda | San Fermin Banter Blitz Cup | Finals
Fb7lJrfVNdQ
videos/chess24media/Fb7lJrfVNdQ already exists.
transcript? False
video? False
loaded: 0

1009 Semi-final | Ja

loaded: 4

1113 Adhiban vs. Flores | San Fermin Banter Blitz Cup | Qualifier C
yL7FsEShdUM
transcript? True
video? True
Couldn't load the video.
loaded: 4

1114 Oparin vs. Piorun | San Fermin Banter Blitz Cup | Qualifier C
XIKuRvLNsuo
transcript? True
video? True
Couldn't load the video.
loaded: 4

1115 Banter Blitz with Peter Svidler
Kqghm08Mcyo
transcript? True
video? True
loaded: 5

1118 Liem Quang Le vs. Robson | San Fermin Banter Blitz Cup | Qualifier B
-RzVtUsLqqU
transcript? True
video? True
Couldn't load the video.
loaded: 5

1120 Banter Blitz with Sam Shankland
TXpIFJJEnDo
transcript? True
video? True
loaded: 6

1122 Banter Blitz with Jan Gustafsson (237)
A22dQozT1ho
transcript? True
video? True
Couldn't load the video.
loaded: 6

1125 Liem Quang Le vs. Santos | San Fermin Banter Blitz Cup | Qualifier B
sAUdpWLDu48
transcript? True
video? True
Couldn't load the video.
loaded: 6

1126 Liem Quang Le vs. Plichta | San Fermin Banter Blitz Cup | Qualifier B
SEtSD4t8hyE
transcript? 

loaded: 73

1435 Banter Blitz with FM Lefong Hua (174)
QEIhUOp6Mw4
transcript? True
video? True
loaded: 74

1436 Banter Blitz with Liem Quang Le
gjx5F16F-fI
transcript? True
video? True
loaded: 75

1439 Banter Blitz with Teimour Radjabov
Wpootf6192g
transcript? True
video? True
loaded: 76

1442 Banter Blitz with FM Lefong Hua (173)
vSLGRHkxCBQ
transcript? True
video? True
loaded: 77

1448 Banter Blitz with Ian Nepomniachtchi
nvwksI6ynbE
transcript? True
video? True
loaded: 78

1452 Blindfold Banter Blitz with Anish Giri
ZAdYrsx-JZ8
transcript? True
video? True
loaded: 79

1454 Banter Blitz with Daniil Dubov
CcFlsnYAGIw
transcript? True
video? True
loaded: 80

1455 Banter Birthday Celebration with Jan and Laurent
WnsqUG6dC9g
transcript? True
video? True
loaded: 81

1457 Banter Blitz with Anish Giri
kzsFV--QlJA
transcript? True
video? True
loaded: 82

1458 Blindfold Banter Blitz with Teimour Radjabov
MWtFrAD6yo0
transcript? True
video? True
loaded: 83

1460 Banter Blitz with FM Lefong Hu

...

Couldn't load the transcript.
Couldn't load the video.
loaded: 1047

5129 Banter Blitz with GM Peter Svidler - January 18, 2017
-4JickOwc-g
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1047

5140 Kingscrusher Banter Blitz - Januar 15, 2017
bgr8fasA5KI
transcript? True
video? True
Couldn't load the transcript.
loaded: 1048

5144 Banter Blitz with Mostrovski - January 13, 2017
Ug9oKy7Pgew
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1048

5146 Banter Blitz with GM Peter Svidler - January 13, 2017
564R-MiqixM
transcript? True
video? True
Couldn't load the transcript.
loaded: 1049

5147 Banter Blitz with IM Christof Sielecki (ChessExplained) - January 11 , 2017
5W4ZKv0g1g4
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1049

5148 Banter Blitz with Jan Gustafsson (58)
T_p1LkFGlK8
transcript? True
video? True
Couldn't load the transcript.
Couldn't load th

Couldn't load the transcript.
Couldn't load the video.
loaded: 1051

5250 Kingscrusher Banter Blitz - October 30, 2016
6ONamSDPnos
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1051

5251 Banter Blitz with IM Christof Sielecki (ChessExplained) - October 26, 2016
OTa-VDC4Ae8
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1051

5253 Kingscrusher Banter Blitz - October 23, 2016
hkwgL-28hHU
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1051

5255 Banter Blitz with Mostrovski - October 21, 2016
xlcBWx10R4k
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1051

5256 Banter Blitz with IM Christof Sielecki (ChessExplained) - October 19, 2016
Z2v5wLE7XGg
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1051

5257 Banter Blitz with Jan Gustafsson (49)
M3un95Klybc
transcript? T

Couldn't load the transcript.
Couldn't load the video.
loaded: 1056

5381 Banter Blitz with ChessExplained - July 20, 2016
f1S6C5kAQCM
transcript? True
video? True
Couldn't load the video.
loaded: 1056

5382 Banter Blitz with Radio Jan
GxAeFd6HU7M
transcript? True
video? True
Couldn't load the video.
loaded: 1056

5383 Banter Blitz with ChessExplained - July 13, 2016
uvnIWntOuiY
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1056

5384 Banter Blitz with Master Ostrovski - August 3, 2016
4WJTWMU3pqY
transcript? True
video? True
Couldn't load the transcript.
loaded: 1057

5389 Banter Blitz with IM Andrey Ostrovskiy - July 27, 2016
zyMmlSVeJtg
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1057

5392 Banter Bullet with Jan Gustafsson
5vndHn6X81I
transcript? True
video? True
Couldn't load the video.
loaded: 1057

5393 Banter Blitz with Kingscrusher - July 31st, 2016
tytjgtgGRaw
transcript? True
video?

Couldn't load the video.
loaded: 1068

5590 Banter Blitz with IM Christof Sielecki (ChessExplained) - April 7, 2016
oCkmzkTZyfY
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1068

5593 Banter Blitz with Jan Gustafsson (27)
lQvnNa_Cmww
transcript? True
video? True
loaded: 1069

5616 Banter Blitz with GM Loek van Wely
3rpTPIKFkLo
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1069

5660 Premium Banter Blitz tournament with Jan
kGNzbpGIOKo
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1069

5663 Banter Blitz with Jan Gustafsson (26)
piK5KC1D16k
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1069

5667 Banter Blitz with Jan Gustafsson (25)
08dqHe4JrCM
transcript? True
video? True
Couldn't load the transcript.
Couldn't load the video.
loaded: 1069

5669 Miss Strategy and Miss Tactics Banter Blitz
YForbIOB4AM
tra

Couldn't load the transcript.
Couldn't load the video.
loaded: 1086

6036 Banter Blitz with IM Christof Sielecki (ChessExplained)
gBkhfI28xts
transcript? True
video? True
loaded: 1087

6047 Banter Blitz: Jan Gustafsson vs startrex
DNJ5pZspMjo
transcript? True
video? True
loaded: 1088

6048 Banter Blitz: Jan Gustafsson vs InDoMable 2015 03 04
dhAqhdjPT-Y
transcript? True
video? True
loaded: 1089

6049 Banter Blitz: Jan Gustafsson vs JVFerreira
MQZIFcM6pUA
transcript? True
video? True
loaded: 1090

6050 Banter Blitz: Jan Gustafsson vs Koenigsmoerder
fA2i4ZUTCNw
transcript? True
video? True
loaded: 1091

6051 Banter Blitz: Jan Gustafsson vs LeQlou
DqkC7AqO254
transcript? True
video? True
loaded: 1092

6052 Banter Blitz: Jan Gustafsson vs sliwa
dAmgBNaT2fA
transcript? True
video? True
loaded: 1093

6056 Banter Blitz: Jan Gustafsson vs brucelee32
SlYnXnP01VQ
transcript? True
video? True
loaded: 1094

6057 Banter Blitz: Jan Gustafsson vs dominik plassmann
jJocx-LVKFg
transcript? True
video? 